<a href="https://colab.research.google.com/github/AdrianFernandoPenafiel/03MAIR-Algoritmos-de-optimizacion/blob/master/Seminarios/Seminario(plantilla)_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: ADRIAN PEÑAFIEL   <br>
Url: https: https://github.com/AdrianFernandoPenafiel/03MAIR-Algoritmos-de-optimizacion/blob/master/Seminarios/Seminario_Algoritmos.ipynb




                                        

In [0]:
#librerias a ser usadas durante todo el seminario
import pandas as pd
import numpy as np
from datetime import datetime

# **Problema 1:**

>1. Elección de grupos de población homogéneos <BR>

  <font color="blue">**Descripción del problema:** </font><br>

Una productora ganadera nos encarga la tarea de seleccionar grupos de terneros para aplicar 3 tratamientos diferentes. Para cada uno de los tratamientos debemos seleccionar 3 grupos de terneros que sean lo mas homogéneos posible en peso para que en los resultados del tratamiento influya lo menos posible el peso del animal. Disponemos de una población de N animales entre machos y hembras

  
<font color="blue">**Restricciones:** </font><br>

  Aun se desconocen los datos concretos de los animales por lo que debemos trabajar con datos que debemos generar de forma aleatoria. Lo que si nos proporcionan es una plantilla de los datos:

ID | Peso | Sexo
-- | -- | --
1 | 38.7 | M
2 | 37.1 | H
3 | 39.2 | M

Hemos podido averiguar que el peso medio del ternero es 37kg con una desviación estándar de 2.1


<font color="blue">**(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones? ** </font> <br>

El primer punto ha considerar es que entendemos por "Grupo Homogeneo", y en primera instancia podemos asumir que se nos pide dividir la poblacion total de N terneros en los 3 grupos para cada uno de los 3 tratamientos, es decir, dividir en 9 grupos en total, 3 para cada uno de los 3 tratamientos.

Para esto debemos calcular la combinatoria posible de agrupar N elementos del conjunto en grupos de aprox N/9 elementos.

$$ C_i^j = \binom {N}{N/9} $$

Que para el caso de tener N=360 terneros, dividrlos en 9 grupos de 40 terneros cada uno, obtendríamos un total de combinacines posibles de 2.3E+53, lo cual lo combierte en un potencial problema tipo NP




<font color="blue">**¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.**</font> <br>

Para este caso, las restricciones lo limitan el universo de soluciones, por lo cual es el mismo descrito en la pregunta anterior



<font color = "blue">**Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumentalo)**</font><br>


Para la selección de la estructura de datos a ser implementada la solución lo realicé pensando en cual sería la mejor forma de presentar la solución final para de allí partir en la potencial selección. <br>

Pensando de esta manera, la respuesta final al ganadero sería el proveerle de un "listado" en donde finalmente se le diga PARA CADA TERNERO, a que GRUPO pertenece y que TRATAMIENTO aplicar, por esto el que mas se me adecua es un *DataFrame*<br>

Por otra parte, mis habilidades de manejo del lenguaje aún no son las mayores, por lo que la solución al problema originalmente fue implementada en el lenguaje y herramienta de mi especialidad, esto es Excel, y el uso de las hojas de calculo se acoplan muy bien para ser traducidas como *DataFrames* en Phyton.<br>


Requeri usar en la implementación 5 Dataframes que los describo a continuacion:<br>

ID Dataframe | Descripcion | Atributo
-- | -- | --
#0 | Datos Originales Terneros, pesos aleatoreos | 
 | ID Ternero | ID
  | Peso del ternero | Peso
  | Tipo o genero del ternero | Tipo
#1 |Pila de trabajo para extraer conunto de terneros mas optimo a cada grupo (voraz) | 
 | ID Ternero | ID
  | Peso del ternero | Peso
  | Desviacion Std, del grupo contiguo. Std(Peso) siguientes T registros, en donde T es el tamaño de grupo calculado (MEJORAR HOMOGENEIDAD INTRAGRUPO)| DesvStd
  |Promedio Peso, del grupo contiguo. Avg(Peso) siguientes T registros, en donde T es el tamaño de grupo calculado|PromGrupo
  |Promedio del Peso del Universo de Terneros|	PromUniv
  |Desviación STD del Promedio del Peso del Grupo vs Promedio Universo de Terneros (MEJORAR HOMEGENEIDAD INTERGRUPOS)	|DesvVsUniv
|Funcion Objetivo a evaluar para seleccionar que conjunto de terneros son mas homogeneos para asignarlos a un grupo, mientras menor sea (AVG(DesvStd,DesvVsUniv) considero un grupo mas Homogeneo Internamente y que lo será vs el resto de grupos	|Funcion
#2|Datos finales objetivo del algoritmo
|ID de tramiento a ser colocado (1,2,3)	|IDTratamiento
|ID Ternero	|ID
|Peso Ternero	|Peso
|Numero de grupo asignado al ternero	|Grupo
#3|Pila de trabajo para extraer conjunto de grupos optimos a cada tratamiento (voraz)|
|Numero de grupo extraido	|Grupo
|Numero de terneros incluidos en el grupo	|Contador
|Desviación STD del peso de los terneros del grupo	|STD_PESO
|Promedio del peso de los terneros del grupo	|PROM_PESO
|Promedio de la Desviacion STD de los 3 grupos contiguos - Ordenados previamente por DESVSTD, HOMOGENEIDAD INTRA-GRUPO	|Prom(STD)
|Desviacion STD del Promedio  de los 3 grupos contiguos - Ordenados previamente por DESVSTD, HOMOGENEIDAD INTER-GRUPO similar pesos promedios	|STD(Prom)
|Funcion Objetivo a evaluar para seleccionar que conjunto de grupos son mas homogeneos para asignarlos a un tratamiento, mientras menor sea (Prom(STD)+STD(Prom)) considero los grupos mas Homogeneos para un mismo tratamiento	|Funcion
#4|	Pila de trabajo para asignar tratamientos a los grupos
|Numero de grupo extraido	|Grupo
|Desviación STD del peso de los terneros del grupo	|Std_Peso
|Promedio del peso de los terneros del grupo	|Prom_Peso
|ID de tramiento a ser colocado (1,2,3)	|IDTratamiento
#5|Resumen de caractarísticas de los grupos asignados al tratamiento
|ID de tramiento a ser colocado (1,2,3)	|Tratamiento
|Promedio de la Desviacion STD de los 3 grupos contiguos - Ordenados previamente por DESVSTD, HOMOGENEIDAD INTRA-GRUPO	|Prom(STD)
|Desviacion STD del Promedio  de los 3 grupos contiguos - Ordenados previamente por DESVSTD, HOMOGENEIDAD INTER-GRUPO similar pesos promedios	|STD(Prom)
|Funcion Objetivo final, mientras menor la suma del valor de esta funcion de los 3 tratamientos mas HOMOGENEIDAD GENERAL DE LA SELECCIÓN	|Funcion









  
 


<font color = "blue"> **Según el modelo para el espacio de soluciones**</font><br>
  
<font color = "blue"> **(*)¿Cual es la función objetivo?**</font><br>

Aqui nuevamente es importante definir que es homogeneidad o que es lo que consideramos como tal. Lo importante es definir 2 tipos de homogeneidad que hay en este problema: <br>

**1. Homgeneidad INTRAGRUPO:** Aquella en la que veremos cuan " homogeneos" son los terneros integrantes de un mismo grupo. Para medir el nivel de esta homogeneidad usaremos la desviación estandar de los pesos de los integrantes del grupo. Como tendremos 9 grupos, el *nivel homogeneo interno de cada uno* de los grupos será su *Desviación Estandar de los Pesos de los integerantes del Grupo*, por lo que el indicador final de Homogeneidad Intragurpo será: **PROMEDIO (DESVIACIONES ESTANDAR DE LOS 9 GRUPOS )** <br>

$$ IndINTRA = promedio(desviacionstdpesos_i)  i=1,9$$


**2. Homgeneidad INTERGRUPO:** Aquella en la que veremos cuan " homogeneos" son los 9 grupos seleccionados. Para medir el nivel de esta homogeneidad usaremos la desviación estandar de los pesos de cada grupo. De igual manera al tener 9 grupos, tratamos a esto como un nuevo conjunto de elementos que debemos calcular su similitud, por los que el indicador final de Homogeneidad Intergurpo será: **DESVIACION ESTANDAR ( PROMEDIOS DE PESO DE LOS 9 GRUPOS )** <br>

$$ IndINTER = desvstd(promediopesos_i)  i=1,9$$

Con esto la funcion objetivo final será al definir:

$$ IndTOTAL = IndINTRA + IndINTER $$

<br>
<font color = "blue"> **(*)¿Es un problema de maximización o minimización?**</font><br>(*)¿Es un problema de maximización o minimización?**</font><br>

Este será un problema de **MINIMIZACION**, ya que un grupo homogeneo perfecto será aquel cuya DESVSTD de los pesos de sus miembros tienda a CERO, de igual forma un conjunto de 9 grupos perfectamente homogeneos serán aquellos cuyos pesos promedios sean iguales y para que esto suceda, la DESVSTD de los *promedios de peso* de los grupos debe tender a cero.



<font color = "blue"> **Diseña un algoritmo para resolver el problema por fuerza bruta**</font><br>

<font color = "blue"> **Calcula la complejidad del algoritmo por fuerza bruta**</font><br>

<font color = "blue"> **(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta**</font><br>

El algoritmo que he decidido implementar se basa en las tecnicas de **ALGORITMO VORAZ** <br>

1. Algortimo Voraz para armado de grupos lo mas homogeneos posibles<br>

N=numero de terneros<br>
TamanoGrupo = N/9<br>
PromedioPesoUniversal = promedio(peso) de todos los registros

for i=1 to 9
>-Ordenar los registros por el peso de los terneros<br>
>for j=1 to N-TamanoGrupo
>> -Calcular Desviacion Estandar del atributo peso de los siguientes TamanoGrupo registros y almacenar en cada registro<br>
-Calcular Promedio del atributo peso de los siguientes TamanoGrupo registros y almacenar en cada registro<br>

>next j<br>
>-Buscar el registro con la menor DesvSTD entre el Promedio del Peso de los *TamanoGrupo* versus el *PromedioPesoUniversal*  registros contiguos , marcara el inicio del grupo de TamanoGrupo registros MAS similar al PormedioUniversal<br>
>-Extraer (borrar de una tabla e insertar en otra) los TamanoGrupo registros iniciando desde el menor marcado hacia el dataframe de registros agrupados, registrar con numero de grupo = i<br>
>-Insertar en tabla de resumen de grupos el registro del grupo, numero, promedio de pesos y desviacion std
>-N = N - TamanoGrupo, nuevo numero de registros


next i

Con esta parte hemos buscado uno a uno el "MEJOR" grupo posible, buscando el grupo contiguo (ordenado) con mejor desviación estandar, asegurando la MEJOR HOMOGENEIDAD INTRAGRUPO y cuyo promedio de peso sea el mas cercano al promedio universal para buscar homogeneidad INTERGRUPOS.<br>
Hasta aqui tenemos los registros marcados y asignados un GRUPO<br>


2. Algortimo Voraz para asignacion de tratamientos a los grupos<br>

NT=numero de tratamientos = 3<br>
GruposxTratamiento = NT/3 = 3<br>

for i=1 to 3
>-Ordenar los registros de los grupos por el peso de los terneros<br>
>for j=1 to NT-GruposxTratamiento
>> -Calcular Desviacion Estandar del atributo peso promedio de los siguientes GruposxTratamiento registros y almacenar en cada registro<br>

>next j<br>
>-Buscar el registro con la menor DesvSTD de los pesos promedios de los registros contiguos , marcara el inicio del mjor grupo para el primer tratamiento<br>
>-Extraer (borrar de un dataframe ye insertar en el otro) los GruposxTratamiento registros iniciando desde el menor marcado hacia el dataframe de registros agrupados, registrar con numero de tratamiento = i<br>
>-Insertar en tabla de resumen de tratamientos el registro del grupo, numero, DESVSTD()promedio de pesos) y PROM()desviacion std)
>-NT = NT - GruposxTratamiento, nuevo numero de registros

next i

Con esta parte hemos buscado uno a uno el "MEJOR" conjunto de 3 grupos posible, asignando de 3 en 3 a los tratamientos y obteniendo una tabla en donde para cada tratamiento estan asignados sus 3 grupos<br>

Finalmente hacemos los JOINS necesarios para marcar a cada registro de Ternero en base a su grupo el tratamiento a colocar<br>



<font color = "blue"> **(*)Calcula la complejidad del algoritmo **</font><br>

Respuesta

<font color = "blue"> **Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**</font><br>

In [0]:
#Crear los df de trabajo en el algoritmo
#las definiciones de estas estructuras fueron descritas en la celda de estructura de datos
df0 = pd.DataFrame(columns=["ID",	"Peso",	"Tipo"]) #Datos originales
df1 = pd.DataFrame(columns=["ID",	"Peso",	"DesvStd",	"PromGrupo",	"PromUniv",	"DesvVsUniv",	"Funcion"])
df2 = pd.DataFrame(columns=["IDTratamiento",	"ID",	"Peso",	"Grupo"])
df3 = pd.DataFrame(columns=["Grupo", "Contador",	"STD_PESO",	"PROM_PESO",	"PromSTD", "STDProm",	"funcion"])
df4 = pd.DataFrame(columns=["Grupo",	"Contador",	"Std_Peso",	"Prom_Peso",	"Tratamiento"])
df5 = pd.DataFrame(columns=["Tratamiento",	"PromSTD",	"STDProm",	"Funcion"])

def CargaDatosTerneros(N):
    
    tipo = ["H", "M"]
    col_id =  list(range(1, N+1)) #ID Ternero
    col_peso = list( np.around( np.random.normal( 37, 2.1, N ), 1 ) ) #Peso del ternero segun datos dados
    col_tipo = list(np.random.choice(tipo, N, p=[0.5, 0.5])) #Genero del Ternero

    df = pd.DataFrame({"ID":col_id,  "Peso":col_peso, "Tipo": col_tipo}, index=col_id) #DataFrame #0, ver descripcion columnas en la celda que describe la estructura de datos
    return df

#Cargar en el dataframe de trabajo inicial, alli se aplicara el primer algoritmo voraz
#df0 = CargaDatosTerneros(27)
#df0.head(5)


<font color = "blue"> **Aplica el algoritmo al juego de datos generado**</font><br>

In [192]:

def AlgoritmoVorazCrearGrupos(NT):
      
        global df0
        global df1
        global df2
        global df3

        Peso1 = 0.5       #Para ponderar la relevacia de los dos indicadores, para el ejemplo doy igual relevancia (peso) al indicador INTRA e INTER
        Peso2 = 1-Peso1   

        NumGrupos = 9 #Dato
        NElementos = NT #Dato
        df0 = CargaDatosTerneros(NElementos) #Genero datos aleatorios y cargo el primer dataframe
        
        df1 = pd.DataFrame(columns=["ID",	"Peso",	"DesvStd",	"PromGrupo",	"PromUniv",	"DesvVsUniv",	"Funcion"])
        df1 = pd.concat([df1,df0])

        df2 = pd.DataFrame(columns=["IDTratamiento",	"ID",	"Peso",	"Grupo"])  #df con registros y grupos
        df3 = pd.DataFrame(columns=["Grupo","Contador",	"STD_PESO",	"PROM_PESO",	"PromSTD", "STDProm",	"funcion"])
       
        PromedioUniversoinicial = df0["Peso"].mean() #Calculo promedio de la poblacion, me sirve como referencia para seleccionar grupos mas homogeneos
        TamanoGrupo = int(round(NElementos / NumGrupos))         #Mejor Tamaño Grupo Inicial
        #print ("Tamaño Inicial Grupo= ",TamanoGrupo )
        contadorRegistrosExtraidos = 0
        #Ordeno los registros para asegurar DSEVSTD optimas 
        df1 = df1.sort_values(by = "Peso")
        df1=df1.reset_index()
        df1=df1.drop(["index","Tipo"],axis=1) #elimino columnas no usadas
        for iGrupo in range(1,NumGrupos+1):
          minGrupo = 999999
          minIndex = -1
          #print ("Grupo: " , iGrupo)
          for jBarrerGrupo in range(0,NElementos - TamanoGrupo + 1 ): #Calcular columnas calculadas DataSet #1
            #print ("jBarrerGrupo: " , jBarrerGrupo)
            df1.iloc[jBarrerGrupo,5]=df1.iloc[jBarrerGrupo:jBarrerGrupo+TamanoGrupo]["Peso"].mean() #actualizar columna PromGrupo
            df1.iloc[jBarrerGrupo,0]=df1.iloc[jBarrerGrupo:jBarrerGrupo+TamanoGrupo]["Peso"].std() #actualizar columna DesvStd
            df1.iloc[jBarrerGrupo,6]=PromedioUniversoinicial #actualizar columna PromUniv
            df1.iloc[jBarrerGrupo,1]= np.std([df1.iloc[jBarrerGrupo,5] , df1.iloc[jBarrerGrupo,6]],ddof=1) #actualizar la columna DesvVsUniv
            df1.iloc[jBarrerGrupo,2]= df1.iloc[jBarrerGrupo,0]*Peso1 + df1.iloc[jBarrerGrupo,1]*Peso2 #actualizar la columna funcion
            if df1.iloc[jBarrerGrupo,2]<minGrupo: #df1.index[df1["Funcion"]==min(df1["Funcion"])].tolist()
              minGrupo = df1.iloc[jBarrerGrupo,2]
              minIndex = jBarrerGrupo    
          dfTemp = df1.iloc[minIndex:minIndex+TamanoGrupo,[3,4]] #extraer solo las columnas ID y Peso
          dfTemp["Grupo"] = iGrupo
          dfTemp["IDTratamiento"] = 0
          df2 = pd.concat([df2,dfTemp])
          #Insercion completa en DF2

          #Insertar en el DataFrame#3 registros resumen del grupo
          df3Temp = pd.DataFrame(columns=["Grupo","Contador",	"STD_PESO",	"PROM_PESO",	"PromSTD", "STDProm",	"funcion"],index=[iGrupo])
          df3Temp.loc[iGrupo,"Grupo"]=iGrupo
          df3Temp.loc[iGrupo,"Contador"]=TamanoGrupo
          df3Temp.loc[iGrupo,"STD_PESO"]=df1.iloc[minIndex,0] #columna desvSTD del grupo extraido
          df3Temp.loc[iGrupo,"PROM_PESO"]=df1.iloc[minIndex,5] #columna Promedio del grupo extraido
          df3Temp.loc[iGrupo,"PromSTD"]=-1 #Seran calculadas para el algoritmo Voraz de asignacion de grupo
          df3Temp.loc[iGrupo,"STDProm"]=-1 #Seran calculadas para el algoritmo Voraz de asignacion de grupo
          df3Temp.loc[iGrupo,"funcion"]=-1 #Seran calculadas para el algoritmo Voraz de asignacion de grupo

          #print("DF3Temp:",df3Temp)
          df3 = pd.concat([df3,df3Temp]) #Insertar registro resumen de grupos
         
          
          #print("Borrando registros desde=",minIndex,":hasta=",minIndex+TamanoGrupo)
          df1=df1.drop(index=df1.index[df1.iloc[minIndex:minIndex+TamanoGrupo].index],axis=0)
          df1=df1.reset_index()
          df1=df1.drop(["index"],axis=1) 
          contadorRegistrosExtraidos = contadorRegistrosExtraidos + TamanoGrupo
          
          #Actualizar infor de indices
          NumGrupos = NumGrupos - 1
          NElementos = NElementos - TamanoGrupo
          if NumGrupos>0:
              TamanoGrupo = int(round(NElementos / NumGrupos))
          #print("Extraidos:",contadorRegistrosExtraidos,":TamG=",TamanoGrupo)
            
            
def AlgortimoVorazParaTratamiento(NT):

  global df3
  global df4
  
  
  AlgoritmoVorazCrearGrupos(NT)
  #DataFrame2 creado con asignacion de grupos en este punto, falta asignar tratamiento, y dataframe #3 de grupos listo
  #Ordeno el DataFrame #3 en base a la DESV.STD interna de los pesos de los terneros del grupo, asi aseguro menor prom(desv.std) para encontrar mejores grupos para el tratamiento
  df3 = df3.sort_values(by = "STD_PESO")
  df3=df3.reset_index()
  df3=df3.drop(["index"],axis=1) #elimino columnas no usadas
  
  df4 = pd.DataFrame(columns=["Grupo",	"Contador",	"Std_Peso",	"Prom_Peso",	"Tratamiento"])
 
  NumGrupos = 9
  NumGruposXTratamiento = 3
 
  #Inicio algoritmo Voraz, buscando la mejor combinacion de 3 grupos para el Trattamiento 1, y asi sucesivamente
  for iTratamiento in range(1,4):
    MinTratamiento = 99999
    minIndex = -1
    for jBarrerGrupo in range(0,NumGrupos - (NumGruposXTratamiento * iTratamiento)+1 ): 
      df3.iloc[jBarrerGrupo,4]=df3.iloc[jBarrerGrupo:jBarrerGrupo+NumGruposXTratamiento]["STD_PESO"].mean() #actualizar columna PromSTD
      df3.iloc[jBarrerGrupo,5]=df3.iloc[jBarrerGrupo:jBarrerGrupo+NumGruposXTratamiento]["PROM_PESO"].std() #actualizar columna DesvProm
      df3.iloc[jBarrerGrupo,6]= df3.iloc[jBarrerGrupo,4] + df3.iloc[jBarrerGrupo,5] #actualizar la columna funcion
      if df3.iloc[jBarrerGrupo,6]<MinTratamiento: 
        MinTratamiento = df3.iloc[jBarrerGrupo,6]
        minIndex = jBarrerGrupo    
      
    #print ("Minimo: ", MinTratamiento, " en index:",minIndex )
    
    for iGrupo in range(0,3):
      #Insertar en el DataFrame#3 registros resumen del grupo
      df4Temp = pd.DataFrame(columns=["Grupo",	"Contador",	"Std_Peso",	"Prom_Peso",	"Tratamiento"],index=[iGrupo])
      df4Temp.loc[iGrupo,"Grupo"]=df3.iloc[minIndex+iGrupo,0]
      df4Temp.loc[iGrupo,"Contador"]=NumGruposXTratamiento
      df4Temp.loc[iGrupo,"Std_Peso"]=df3.iloc[minIndex+iGrupo,2] #columna desvSTD del grupo extraido
      df4Temp.loc[iGrupo,"Prom_Peso"]=df3.iloc[minIndex+iGrupo,3] #columna Promedio del grupo extraido
      df4Temp.loc[iGrupo,"Tratamiento"]=iTratamiento
      df4 = pd.concat([df4,df4Temp]) #Insertar registro resumen de grupos
      
    #print("Borrando registros desde=",minIndex,":hasta=",minIndex+NumGruposXTratamiento)
    df3=df3.drop(index=df3.index[df3.iloc[minIndex:minIndex+NumGruposXTratamiento].index],axis=0)
    df3=df3.reset_index()
    df3=df3.drop(["index"],axis=1) 



########################################################################
#Llamada Principal para solucion

N_Terneros = 362
AlgortimoVorazParaTratamiento(N_Terneros)

#Actualizo el DF2 la columna tratamiento JOIN por la columna GRUPO
#y alli obtenemos la solucion requerida: DF2: Que tratamiento colocar a cada ID ternero que esta asignado a un grupo
tempjoin = df2.merge(df4,on="Grupo")
tempjoin["IDTratamiento"] = tempjoin["Tratamiento"]

df2 = tempjoin.drop(["Contador","Std_Peso","Prom_Peso","Tratamiento"],axis=1) #Borro columnas extras del JOIN
#Solucion final: Grupo asignado, ID Ternero, ID Tratamiento, Peso del Ternero
df2.sample(10)




,Grupo,ID,IDTratamiento,Peso
64,2,61,2,37.4
240,7,269,2,34.1
39,1,339,1,37.4
359,9,106,3,42.9
176,5,22,1,35.7
221,6,237,2,38.9
63,2,181,2,37.4
168,5,211,1,35.5
334,9,127,3,33.5
261,7,125,2,34.8


<font color = "blue"> **(*)Calcula la complejidad del algoritmo **</font>


La complejidad del algoritmo propuesto llega a ser O(n^2). Debido a que en el peor de los casos se tiene 2 for's anidados.


# **Problema 2:**

>2. Organizar los horarios de partidos de La Liga <BR>

## Descripcion

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios que maximice la audiencia.

Los horarios disponibles se conocen a priori y son los siguientes:

Dia | Horario
--- | ---
Viernes | 20
Sabado | 12, 16, 18, 20
Domingo | 12, 16, 18, 20
Lunes | 20

### Restricciones

* En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores que tienen relación directa con la audiencia. Hay 4 equipos en la categoría A, 10 equipos de categoría B y 6 equipos de categoría C.

* Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

. | Categoria A | Categoria B | Categoria C
--- | --- | --- | ---
Categoria A | 2 millones | 1.3 millones | 1 millon
Categoria B | . | 0.9 millones | 0.75 millones
Categoria C | . | . | 0.47 millon

* Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla:

. | Viernes | Sabado | Domingo | Lunes
--- | --- | --- | --- | ---
12 h | - | 0.55 | 0.45 | -
16 h | - | 0.7 | 0.75 | -
18 h | - | 0.8 | 0.85 | -
20 h | 0.4 | 1 | 1 | 0.4

* Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes




<font color = "blue"> **(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?**</font><br>

Partiendo de la premisa que siempre 1 quipo juega un solo partido en una fecha (jornada viernes-lunes) y que el torneo es todos contra todos, las posibilidades serían el numero de quipos (N) armados de 2 en 2 por partido, por tanto el numero total de posibilidades es la multiplicacion de las COMBINATORIAS de (20,2) x (18,2) x ... x (2,2) el total dividido por 2 y finalmente multiplicado por las 19 fechas. Nuevamente un numero que cataloga al problema de tipo factorial.

<font color = "blue"> **¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.**</font><br>




<font color = "blue"> **Modelo para el espacio de soluciones**</font> <br>

<font color = "blue"> **(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)**</font>


Para encontrar un algoritmo a resolver el problema, decidi tomar la siguiente estrateguia: <br>

1) Construir los partidos de forma tal que se garanticen las 19 fechas de todos contra todos sin repetir las partidas, para esto use la dsitribución ROUND-ROBIN y usando nombres de quipos genéricos, es decir, aun desconozco quienes serán los equipos Eq1, Eq2, Eq3 <br>

2) Una vez construido el calendario de juegos con equipos genericos, el problema a optimizar es asignar quien es cada equipo, o mejor dicho, que **CATEGORIA** debe tener cada equipo EQ1,EQ2,...<br>

Con esto el problema se convierte en encontrar la asignación de **CATEGORIAS** a los equipos que generen la **MAXIMA AUDIENCIA POSIBLE**<br>

Para esto las estructuras básicas utilizadas seran:<br>

1. Dataframes: Para almacenar la informacion dato inicial Equipos, Horarios, AudienciasxCategoria y Partidas<br>
2. Cadena de Caracteres: Una cadena de 20 caracteres que contendra las categorias a asignar a los 20 Equipos<br>

CADENA DE CATEGORIAS:<br>
Para entender esta cadena pongo un ejemplo: " AAAABBBBBBBBBBCCCCCC ", nos dice que el EQ1 es A, el EQ2 es A, y asi cada caracter para cada EQUIPO. Aqui debemos siempre validar  la **RESTRICCION** dada, 4 "A", 10 "B" y 6 "C".


<font color = "blue"> **Según el modelo para el espacio de soluciones**</font><br>
<font color = "blue"> **(*)¿Cual es la función objetivo?**</font><br>

La funcion objetivo será la SUMATORIA DE LA AUDIENCIA NETA = SUMATORIA DE AUDIENCIAS DE CADA PARTIDO ACORDE A SUS CATEGORIAS x FACTOR DE HORARIO

<font color = "blue"> **(*)¿Es un problema de maximización o minimización?**</font><br>
Es un problema de MAXIMIZACION ya que debemos intentar obtener la MAXIMA AUDIENCIA NETA POSIBLE

<font color = "blue"> **Diseña un algoritmo para resolver el problema por fuerza bruta</font> **

<font color = "blue"> **Calcula la complejidad del algoritmo por fuerza bruta**</font>

<font color = "blue"> **(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta**</font>


La tecnica que he decisidido implementar para este problema es un **ALGORITMO GENETICO**. Considero que se adapta perfectamente a este tipo de problema ya que pequeñas modificaciones en el orden en el cual se enfrentan los equipos generan mejores audiencias. Con esto la generación aleatoria de una poblacion inicial de la cadena de 20 caracteres que representan las categorias de los Equipos se presta a generar las pequeñas mutaciones que van mejorando la audiencia progresivamente hasta alcanzar puntos de estabilidad.


Algoritmo Propuesto:

1) GenerarPartidas #round robin fijo con equipos genericos EQ1, EQ2,..EQ20 <br>
2) DefinirParametrosgenerales #Numero de generaciones que se usaran, Numero de Hijos por mutacion, Numero de hijos aleatorios<br>
3) DefinirCriterioPararGeneraciones # Numero de generaciones en las cuales ya no se obtienen mejoras<br>
4) CrearPoblacionInicialCromosomas #Cadenas de 20 caracteres aleatorias<br>
5) Inicializar contadores de generaciones <br>


do<br>

> 6) EscogerMejoresCromosomas #calcular audiencias netas<br>
7) CrearNuevosHijosPorMutacion<br>
8) CrearNuevosHijosAleatorios<br>
9) CalcularMayorAudienciaEncontrada<br>



> Si NuevaMayorAudiencia supera AudienciaMayorAnterior entonces
             
             GrabarNuevaMayorAudiencia
             
 > Casocontrario
             
             GeneracionesEstables += 1
 
             
> ContadorGeneracion += 1

Loop Until GeneracionesEstables alcnzadas O GeneracionesLimite alcanzadas





<font color = "blue"> **(*)Calcula la complejidad del algoritmo**</font>

La complejidad del **ALGORITMO GENETICO** propuesto es del orden O (n log n), ya que en el peor de los escenarios se manejarán un ciclo que controla la generación de las nuevas combinaciones y la aplicación en ciclos del calculo de la AUDIENCIA generada por cada nuevo "cromosoma" en un calendario previamente ya construido y definitivamente dependera el LIMITE de generaciones y de mutaciones en cada generación que establezcamos




<font color = "blue"> **Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**</font>

Para la solucion propuesta no se requiere la generación de Datos de Entrada aleatorios, ya que estan definidos os 20 equipo y será parte del algoritmo la generación inicial aleatoria de la combinacion de categorias propuesta.

<font color = "blue">**Aplica el algoritmo al juego de datos generado**</font>

Por temas de tiempo, el **ALGORITMO FUE IMPLEMENTADO EN EXCEL**, por favor mirar adjuntos.<br>
Se ha alcanzado un ESCENARIO de combinacion de categorias optimo = "ACCCBBBCCABABBBBBCBA", con la cual se alcanza una audiencia de 135,32 millones de audiencia neta

Option Explicit
Const numEquipos As Integer = 20



Sub GenerarPartidos()
    'Algoritmo Round-Robin para generar todos los partidos de las 19 fechas
    'Que asegura un partido por cada fecha para cada equipo
    'Genera los registros iniciales del DataSet #4
    
    Dim colv1 As Integer
    Dim colv2 As Integer
    Dim NumPartidosxFecha As Integer
    Dim NumFechas As Integer
    
    Dim numf As Integer
    Dim numP As Integer
    Dim fila As Integer
    
    Dim i As Integer
    Dim t1 As Integer
    Dim t2 As Integer
    
    colv1 = 15 'Para Equipo1
    colv2 = 16 'Para Equipo2
    
    NumPartidosxFecha = Int(numEquipos / 2)
    NumFechas = numEquipos - 1
    numf = 1
    numP = 1
    fila = 3
    For i = 1 To 190
        'Primera fecha es setup inicial
        If numf = 1 Then
            t1 = numP
            t2 = numEquipos + 1 - numP
            Cells(fila, 14) = numf
            Cells(fila, colv1) = t1
            Cells(fila, colv2) = t2
        End If
        
        If numf > 1 Then
            Cells(fila, 14) = numf
            
            t2 = Cells(fila - NumPartidosxFecha, colv2) - 1 'Rota en base a registros previos
            t2 = IIf(t2 < 2, numEquipos, t2) 'Asegura rotacion ...19,20,1,2...
            Cells(fila, colv2) = IIf(t2 < 2, numEquipos, t2)
            
            If numP = 1 Then 'El primer partido siempre Eq1, equipo pivot
                t1 = 1
            Else
                If numP = 2 Then 'Asegura rotacion ...19,20,1,2...
                    t1 = Cells(fila - 1, colv2) + 1
                Else
                    t1 = Cells(fila - 1, colv1) + 1
                    t1 = IIf(t1 > numEquipos, 2, t1)
                End If
            End If
            Cells(fila, colv1) = t1
        End If
        
        numP = numP + 1
        If numP = NumPartidosxFecha + 1 Then 'Termino la fecha
            numP = 1
            numf = numf + 1
        End If
        Cells(fila, 17) = "" 'La etiqueta sera cargada con el mejor escenario encontrado
        
        fila = fila + 1
    Next i
End Sub
Sub ColocarEtiquetasPartidas()
    Dim i As Integer
    Dim fila As Integer
    'Actualizar etiquetas de partidos en en Dataset#4
    fila = 3
    For i = 1 To (numEquipos - 1) * Int(numEquipos / 2)
        Cells(fila + i - 1, 17).FormulaR1C1 = "=VLOOKUP(RC[-2],R3C1:R22C2,2,FALSE) & "" vs ""&VLOOKUP(RC[-1],R3C1:R22C2,2,FALSE)"
    Next i
End Sub

Function CalculaAudienciaCamposOrdenarPartidos() As Double
    Dim fila As Integer
    Dim i As Integer
    Dim filaNumHorario As Integer
    Dim aN As Double
    
    'Asume que el DataSet#1 esta lleno con datos de categorias
    'Ejecuta los joins necesarios para tener audiencias brutas
    fila = 3
    For i = 1 To 190
        'Join con dataset#1 para generar Categoria Partido
        Cells(fila + i - 1, 18).FormulaR1C1 = "=VLOOKUP(RC[-3],R3C1:R22C3,3,FALSE) & VLOOKUP(RC[-2],R3C1:R22C3,3,FALSE)"
        Cells(fila + i - 1, 18).Value = Cells(fila + i - 1, 18).Value
        'Join con dataset#2 para traer audiencia bruta
        Cells(fila + i - 1, 19).FormulaR1C1 = "=VLOOKUP(RC[-1],R3C5:R11C6,2,FALSE)"
        Cells(fila + i - 1, 19).Value = Cells(fila + i - 1, 19).Value
    Next i

    'Ordenar el dataset#4 por las columnas
    '1) Fecha ascendente
    '2) Audiencia bruta descendente
    
    Range("S2").Select
    Range(Selection, Selection.End(xlToLeft)).Select
    Range(Selection, Selection.End(xlDown)).Select
    ActiveWorkbook.Worksheets("Problema_2_Solucion").Sort.SortFields.Clear
    ActiveWorkbook.Worksheets("Problema_2_Solucion").Sort.SortFields.Add2 Key:=Range("N3:N192") _
        , SortOn:=xlSortOnValues, Order:=xlAscending, DataOption:=xlSortNormal
    ActiveWorkbook.Worksheets("Problema_2_Solucion").Sort.SortFields.Add2 Key:=Range("S3:S192") _
        , SortOn:=xlSortOnValues, Order:=xlDescending, DataOption:=xlSortNormal
    With ActiveWorkbook.Worksheets("Problema_2_Solucion").Sort
        .SetRange Range("N2:S192")
        .Header = xlYes
        .MatchCase = False
        .Orientation = xlTopToBottom
        .SortMethod = xlPinYin
        .Apply
    End With
    
    'Una vez ordenado, asigna horarios de mayor a menor
    'Asi aseguramos maximizar audiencia bruta por orden
    'Asume que el dataset#3 esta ordenado
    '   Factor, descendente como dato inicial
    
    fila = 3
    filaNumHorario = 3 'Fijos 10 horarios
    aN = 0#
    
    For i = 1 To 190
        Cells(fila + i - 1, 20).Value = Cells(filaNumHorario, 8) 'Horario asignado de mayor a menor factor
        'Join con dataset#3 para traer factor de hora para calcular audiencia neta
        Cells(fila + i - 1, 21).FormulaR1C1 = "=VLOOKUP(RC[-1],R3C8:R12C9,2,FALSE)"
        Cells(fila + i - 1, 21).Value = Cells(fila + i - 1, 21)
        Cells(fila + i - 1, 22).FormulaR1C1 = "=RC[-1]*RC[-3]" 'Calcula Audiencia Neta
        Cells(fila + i - 1, 22).Value = Cells(fila + i - 1, 22).Value
        aN = aN + Cells(fila + i - 1, 22).Value 'acumulo audiencia neta
        filaNumHorario = IIf(filaNumHorario < (10 + 2), filaNumHorario + 1, 3) 'Barre el dataset#3
        
    Next i
    CalculaAudienciaCamposOrdenarPartidos = aN
End Function

Sub CargaEquipos(catCadena As String)
    Dim fila As Integer
    Dim i As Integer
    Dim cat As String
    
    Dim contadorA As Integer
    Dim contadorB As Integer
    Dim contadorC As Integer
    
    Dim cont As Integer
    'Carga inicial DataSet#1
    fila = 3
    'Carga inicial categorias
    contadorA = 0
    contadorB = 0
    contadorC = 0
    For i = 1 To 20
        cat = Mid(catCadena, i, 1)
        Cells(fila + i - 1, 1) = i 'ID equipo
        Cells(fila + i - 1, 3) = cat 'Categoria
        If cat = "A" Then
            contadorA = contadorA + 1
            cont = contadorA
        End If
        If cat = "B" Then
            contadorB = contadorB + 1
            cont = contadorB
        End If
        If cat = "C" Then
            contadorC = contadorC + 1
            cont = contadorC
        End If
        Cells(fila + i - 1, 2) = "Eq_" & cat & "_" & cont 'ID equipo
    Next i
End Sub

Sub InsertarRegistroEscenario(cadena As String, audiencia As Double)
    Dim maxID As Long
    Dim fila As Long
    'inserta Registro de Escenario Generado
    'dataset #5
    fila = 3
    'buscar fila final para insertar registro
    maxID = 0
    While Cells(fila, 25).Value <> ""
        If Cells(fila, 25).Value > maxID Then
            maxID = Cells(fila, 25).Value
        End If
        fila = fila + 1
    Wend
    'Insertar
    Cells(fila, 25) = maxID + 1 'ID combinacion
    Cells(fila, 26) = cadena 'Combinacion
    Cells(fila, 27) = audiencia 'aUDICIENCA
    
End Sub
Function ObtenerCadenaAleatoria() As String
    'Generar una cadena de CAT aleatoria, en Excel lee columna AD
    Dim cadena As String
    Dim fila As Integer
    Dim gen As String
    Dim pos_gen As Integer
    cadena = "BBBBBBBBBBBBBBBBBBBB" 'Cadena Default
    fila = 3
    Do
        gen = Cells(fila, 29).Value
        pos_gen = Cells(fila, 30).Value
        If gen <> "" Then
            'reemplazar gen
            cadena = Mid(cadena, 1, pos_gen - 1) & gen & Mid(cadena, pos_gen + 1, numEquipos - pos_gen)
        End If
        fila = fila + 1
    Loop Until gen = ""
    ObtenerCadenaAleatoria = cadena
End Function

Function EvolucionarPorMutacion(cadena As String) As String
    Dim posDesde As Integer
    Dim posHacia As Integer
    Dim numGenes As Integer
    Dim i As Integer
    Dim cadEvol As String
    Dim genDesde As String
    Dim genHacia As String
    Dim auxIntercambio As String
    'Realizar una ligera mutacion de orden para mantener buenas audiecias
    'Intercambiar ciertas subcadenas
    cadEvol = cadena
    While cadEvol = cadena 'Asegurar que la mutacion sea diferente
        numGenes = Int((3 * Rnd()) + 0) '1 a 2 genes
        posDesde = Int((8 * Rnd()) + 1) '1 a 8 pos_inicial
        posHacia = Int((7 * Rnd()) + 8) '8 a 14 pos_final
        For i = 1 To numGenes
            genDesde = Mid(cadEvol, posDesde, 1)
            genHacia = Mid(cadEvol, posHacia, 1)
            'Escribir Gen(i) Hacia -> Desde
            cadEvol = Mid(cadEvol, 1, posDesde - 1) & genHacia & Mid(cadEvol, posDesde + 1, numEquipos - posDesde)
            'Escribir Gen(i) Desde -> hacia
            cadEvol = Mid(cadEvol, 1, posHacia - 1) & genDesde & Mid(cadEvol, posHacia + 1, numEquipos - posHacia)
            posDesde = posDesde + 1 'Recorrer posicion siguiente gen
            posHacia = posHacia + 1 'Recorrer posicion siguiente gen
        Next i
    Wend
    EvolucionarPorMutacion = cadEvol
End Function

Sub EscogerMejoresCromosomas(num As Integer, total As Integer)
    Dim filaABorrar As Integer

    'Aqui se dejara en el DataSet#5 los registros que contengan los (NUM) cromosomas con las mejores audiencias
    'Ordenar por mejor audiencia
    Range("Y3:AA" & 3 + total + 10).Select
    ActiveWorkbook.Worksheets("Problema_2_solucion").Sort.SortFields.Clear
    ActiveWorkbook.Worksheets("Problema_2_solucion").Sort.SortFields.Add2 Key:= _
        Range("AA3"), SortOn:=xlSortOnValues, Order:=xlDescending, DataOption:= _
        xlSortNormal
    With ActiveWorkbook.Worksheets("Problema_2_solucion").Sort
        .SetRange Range("Y3:AA" & 3 + total + 10)
        .Header = xlGuess
        .MatchCase = False
        .Orientation = xlTopToBottom
        .SortMethod = xlPinYin
        .Apply
    End With
    'Solo Dejar los TOP (num) enviado como parametro
    'Borrar los ultimos registros NO TOP del dataset#5
    filaABorrar = 3 + num
    While Cells(filaABorrar, 25).Value <> ""
        Range("Y" & filaABorrar & ":AA" & filaABorrar).Select
        Range("AA" & filaABorrar).Activate
        Selection.Delete Shift:=xlUp
    Wend
End Sub

Sub Test()
    Dim audiencia As Double
    Dim cadena As String
    Dim numGeneraciones As Long
    Dim numGeneracionesEstables As Long
    Dim MaxnumGeneracionesEstables As Long
    Dim numHijosPorMutacion As Integer
    Dim numHijosRandom As Integer
    Dim MejorAudienciaGeneracion As Double
    Dim i As Long
    Dim j As Long
    Application.ScreenUpdating = False
    Randomize (100)
    'Configurar Parametros Evolucion
    numHijosPorMutacion = 5 'Top cromosomas que generan mejor audiencia y que seran seleccionados para mutar
    numHijosRandom = 25 'Buscar manera de insertar nuevos escenarios
    numGeneraciones = 100 'Numero Maximo de Generaciones
    MaxnumGeneracionesEstables = 20 'Numero de control de salida cuando la maximización se estabiliza
    
    'Generar Partidos Round Robin Iniciales
    GenerarPartidos
    
    'Crear Primera Poblacion de cromosomas
    For i = 1 To numHijosRandom
        cadena = ObtenerCadenaAleatoria
        CargaEquipos cadena
        'Calcula campos
        audiencia = CalculaAudienciaCamposOrdenarPartidos
        'Inserta Registro de escenario obtenido
        InsertarRegistroEscenario cadena, audiencia
        'Debug.Print "Poblacion inicial generado: " & i
    Next i
    
    'Control de detencion definido por dato numero de Generaciones en los parametros
    i = 1: numGeneracionesEstables = 0
    Do

        
        EscogerMejoresCromosomas numHijosPorMutacion, numHijosPorMutacion * 2 + numHijosRandom
        
        'Crea nueva poblacion por mutaciones de mejores cromosomas
        For j = 1 To numHijosPorMutacion
            cadena = EvolucionarPorMutacion(Cells(3 + j - 1, 26).Value)
            'Procesar nuevo hijo
            CargaEquipos cadena
            audiencia = CalculaAudienciaCamposOrdenarPartidos
            InsertarRegistroEscenario cadena, audiencia
            'Debug.Print "Generacion# " & i & ": Hijo mutacion mejor cromosoma#: " & j
        Next j
        
        'Crea nueva poblacion aleatoria
        For j = 1 To numHijosRandom
            cadena = ObtenerCadenaAleatoria
            'Procesar nuevo hijo
            CargaEquipos cadena
            audiencia = CalculaAudienciaCamposOrdenarPartidos
            InsertarRegistroEscenario cadena, audiencia
            'Debug.Print "Generacion# " & i & ": Hijo aleatorio #: " & j
        Next j
        'Control de estabilidad de solucion maxima
        If (Cells(2, 28).Value = MejorAudienciaGeneracion) Then
            numGeneracionesEstables = numGeneracionesEstables + 1
        Else
            numGeneracionesEstables = 0
        End If
        MejorAudienciaGeneracion = Cells(2, 28).Value 'Max audiencia en el Dataset#5
        Debug.Print "Generacion# " & i & " Mejor Audiencia alcanzada: " & MejorAudienciaGeneracion & " : GeneracionesEstables: " & numGeneracionesEstables
        i = i + 1
    Loop Until (i = numGeneraciones) Or (numGeneracionesEstables = MaxnumGeneracionesEstables)
    
    'Cargar Mejor Cromosoma encontrado
    cadena = Cells(3, 26).Value 'Lectura maxima audiencia Dataset#5
    'Procesar nuevo hijo
    CargaEquipos cadena
    audiencia = CalculaAudienciaCamposOrdenarPartidos
    ColocarEtiquetasPartidas
    'Dataset#4 con resultado final optimo cargado
    
    Application.ScreenUpdating = True
End Sub




<font color = "blue">**Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo**</font>

La principal fuente de ayuda ha sido la forma de generar un caledario generico de TODOS contra TODOS en un torneo <br>

https://es.wikipedia.org/wiki/Sistema_de_todos_contra_todos

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

# **Problema 3:**

<font color = "blue">**3. Combinar cifras y operaciones**</font> <BR>

- Disponemos de las 9 cifras del 1 al 9 (excluimos el cero)
- 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
- Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una
cantidad dada. Un ejemplo sería para obtener el 4:
$$4+2-6/3*1 = 4$$
- Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las
siguientes cuestiones:
  - ¿Qué valor máximo y mínimo a priori se pueden obtener según las condiciones?
  
  Se lo calcula fácilmente realizando la multiplicacion máxima disponible (8*9), para el minimo colocarlo en negativo, para el máximo colocarlo en positivo de alli colocar la minima division posible (2/1) como inicial y acorde al signo de la multiplicacion sumar el siguiente digito mas grande 7
  
  $$ Minimo=-69, Maximo =77 $$
  
  
  - ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo ?
  
  $$ Si $$

<font color = "blue">**(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?**</font><br>


Tenemos 9 elementos a combinar DIGITO1 SIGNO1 DIGITO 2 SIGNO 2 ... DIGITO4 SIGNO 4 DIGITO 5 <br>

Por lo que todas las combinaciones será el resultado de la siguiente multiplicacion:

$$ 9*4*9*4*9*4*9*4*9 $$



<font color = "blue">**¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones**</font><br>

Las dos restricciones son la no repeticion de los digitos y de los signos, por lo que las posibilidades serían:

$$ 9*4*8*3*7*2*6*1*5 $$



<font color = "blue">**Modelo para el espacio de soluciones**</font><br>
<font color = "blue">**(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)**</font><br>


Las estructuras utilizadas son DOS ARREGLOS, el primero de 5 celdas para los digitos, y el segundo de 4 CELDAS para los 4 signos posibles

<font color = "blue">**Según el modelo para el espacio de soluciones**</font><br>
<font color = "blue">**(*)¿Cual es la función objetivo?**</font><br>

Encontrar el valor ENTERO dado como objetivo

<font color = "blue">**(*)¿Es un problema de maximización o minimización?**</font><br>

Es de minimización, ya que lo que se busca es minimizar a CERO la diferencia entre el valor objetivo dado y el resultado de la operacion propuesta

<font color = "blue">**Diseña un algoritmo para resolver el problema por fuerza bruta **</font><br>

El algoritmo de fuerza bruta seria un programa de 9 FOR's anidados para variar cada DIGITO y cada SIGNO

<font color = "blue">**Calcula la complejidad del algoritmo por fuerza bruta **</font><br>

O(n^9) debido a la anidación de los FOR´s

<font color = "blue">**(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta **</font><br>

He decidido aplicar la técnica de **DIVIDE y VENCERAS** <br>

El primer paso es buscar la operación MATEMÁTICA DE MULTIPLICACION que mas me acerque al resultado posible, esto lo logro obteniendo el FACTOR1 y FACTOR2, para esto inicio colocando como FACTOR1 el valor entero de la raiz cuadrada del valor absoluto del valor objetivo, y para FACTOR2 = FACTOR1 +1  <br>

La operacion MULTIPLICACION siempre estará al final de la expresion, con esto <br>
DIGITO 4 <- FACTOR 1<br>
SIGNO 4 <- "x"<br>
DIGITO 5 <- FACTOR 2<br>

Una vez determinada la OPERACION MULTIPLICACION INICIAL, defino el orden de los signos para la operación final definimos:<br>

Siempre empezaremos con la DIVISION. <br>
SIGNO 1 <- "/"

Si el valor objetivo es negativo, entonces el componente de multiplicacion debe ser negativo:<br>
SIGNO 3 <- "-"<br>
SIGNO 2 <- "+"<br>

Dejando como opción de SIGNOS para la expresión final<br>


Si el valor objetivo es positivo, entonces el componente de multiplicacion debe ser positivo:<br>
SIGNO 3 <- "+"<br>
SIGNO 2 <- "-"<br>

Una vez definido esto, solo resta iterar sobre los DIGITOS 1, 2 y 3 hasta llegar al valor requerido. 



<font color = "blue">** (*)Calcula la complejidad del algoritmo **</font><br>

Se logra reducir a una complejidad O(n^4) ya que con el algortimo propuesto se juega solo entre 3 DIGITOS y dos combinaciones de SIGNOS

<font color = "blue">**Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**</font><br>

No es necesario dada la tecnica usada

<font color = "blue">**Aplica el algoritmo al juego de datos generado**</font><br>

Por temas de tiempo, el **ALGORITMO FUE IMPLEMENTADO EN EXCEL**, por favor mirar adjuntos.<br>


Option Explicit
Dim num_iteraciones As Integer
Dim num_acum_iteraciones As Integer

Function es_solucion(solucionDigitos As Variant, solucionSignos As Variant, valorObj As Integer)
    Dim strEval As String
    Dim i As Integer
    
    num_iteraciones = num_iteraciones + 1
    strEval = "="
    For i = 1 To 5
        strEval = strEval & solucionDigitos(i)
        If i < 5 Then
            strEval = strEval & solucionSignos(i)
        End If
    Next i
    Cells(1, 1).FormulaR1C1 = strEval
    es_solucion = (valorObj = Cells(1, 1).Value) 'Usar eval de Excel
End Function

Function GrabarSolucion() As String
    GrabarSolucion = "'" & Cells(1, 1).FormulaR1C1
End Function


Function EsDigitoValido(solucionDigitos As Variant, solucionSignos As Variant, i As Integer) As Boolean
    'Para ser valido, debe no generar divisiones inexactas y no debe repetirse
    Dim resp As Boolean
    Dim j As Integer
    resp = True
    'buscar en todo el arreglo
    For j = 1 To UBound(solucionDigitos)
        If solucionDigitos(j) = solucionDigitos(i) And j <> i Then
            resp = False
        End If
    Next j
    'buscar que no genere divisiones no enteras
    If resp And (solucionDigitos(1) / solucionDigitos(2)) <> Int((solucionDigitos(1) / solucionDigitos(2))) Then
        resp = False
    End If
        
    EsDigitoValido = resp
End Function


Function FuerzaBrutaSoluciones(ByRef solucionDigitos As Variant, ByRef solucionSignos As Variant, valorObj As Integer) As String
    Dim sol_encontrada As String
    sol_encontrada = ""
    Dim i1 As Integer
    Dim i2 As Integer
    Dim i3 As Integer
    
    i1 = 2: Do 'i1: Numerador para entero solo > 2
        solucionDigitos(1) = i1
            i2 = 1: Do 'i2 Denominador solo 1 hasta 4, resto daria siempre decimales
                solucionDigitos(2) = i2
                    i3 = 1: Do  'Todas las opciones sin restriccion
                        solucionDigitos(3) = i3
                        If EsDigitoValido(solucionDigitos, solucionSignos, 3) Then
                            If es_solucion(solucionDigitos, solucionSignos, valorObj) Then
                              sol_encontrada = GrabarSolucion
                            End If
                        End If
                    i3 = i3 + 1: Loop Until i3 > 9 Or sol_encontrada <> ""  'Next i3
            i2 = i2 + 1: Loop Until i2 > 4 Or sol_encontrada <> ""  'Next i2
    i1 = i1 + 1: Loop Until i1 > 9 Or sol_encontrada <> "" 'Next i1
    FuerzaBrutaSoluciones = sol_encontrada
End Function

Sub Test(valor As Integer)
    Dim resuelto As String
    Dim factor1 As Integer
    Dim factor2 As Integer
    Dim caso_especial As Boolean
    ReDim solucionDigitos(1 To 5)
    ReDim solucionSignos(1 To 4)
    
    caso_especial = False
    num_iteraciones = 0
    'Inicial
    factor1 = Int(Sqr(Abs(valor)))
    factor2 = factor1 + 1
    'Posiciones
    
    'Si valor objetivo >0 entonces requerimos que la multiplicacion sea + y la pondremos al final de la expresion
    solucionDigitos(4) = factor1: solucionDigitos(5) = factor2
    If valor > 0 Then
        solucionSignos(1) = "/": solucionSignos(2) = "-": solucionSignos(3) = "+": solucionSignos(4) = "*"
    End If
    'Si valor objetivo <=0 entonces requerimos que la multiplicacion sea - y la pondremos al medio de la expresion
    If valor <= 0 Then
        solucionSignos(1) = "/": solucionSignos(2) = "+": solucionSignos(3) = "-": solucionSignos(4) = "*"
    End If
    'Valores iniciales 3 digitos faltantes
    solucionDigitos(1) = 2: solucionDigitos(2) = 1: solucionDigitos(3) = 3
    Do
        'Iteraremos solo sobre los digitos no factores de la multiplicacion
        resuelto = FuerzaBrutaSoluciones(solucionDigitos, solucionSignos, valor)
        If resuelto = "" Then
            'Expander factor multiplicacion
            If factor2 < 9 Then
                factor2 = factor2 + 1
                solucionDigitos(5) = factor2
            Else
                If factor1 > 1 Then
                    factor1 = factor1 - 1
                    solucionDigitos(4) = factor1
                Else
                    If Not caso_especial Then
                        'reoganizar las permutaciones de expancion del multiplicador
                        factor1 = Int(Sqr(Abs(valor))) + 1
                        factor2 = factor1 + 1
                        solucionDigitos(4) = factor1
                        solucionDigitos(5) = factor2
                    Else
                        'Llego al limite sin solución
                        resuelto = "Sin solución!!!!"
                        caso_especial = True
                    End If
                End If
            End If
        End If
    Loop Until resuelto <> ""
    Debug.Print "Valor = " & valor & " resuelto en: #" & num_iteraciones & "# iteraciones, formula = #" & resuelto & "#"
    Cells(1, 2).Value = resuelto
End Sub

Sub ComprobarTodos()
    Dim x As Integer
    num_acum_iteraciones = 0
    Application.ScreenUpdating = False
    For x = -69 To 77
        Test x
        num_acum_iteraciones = num_acum_iteraciones + num_iteraciones
    Next x
    Application.ScreenUpdating = True
    MsgBox "Soluciones para rango [-69,77] encontradas en " & num_acum_iteraciones & " validaciones de formula", vbInformation
    Cells(3, 1).Value = num_acum_iteraciones
    
End Sub

Sub ComprobarUno()
    Dim valorObjetivo As Integer
    Application.ScreenUpdating = False
    
    valorObjetivo = InputBox("Por favor ingrese el numero entero [-69,77] a buscar", "Ingresar Valor Objetivo", 0)
    If valorObjetivo > 77 Or valorObjetivo < -69 Then
        MsgBox "El valor debe ser entre -69 y 77", vbCritical
    End If
    Test valorObjetivo
    
    Application.ScreenUpdating = True
    
    MsgBox "Solucion: " & Right(Trim(Cells(1, 2).Value), Len(Cells(1, 2).Value) - 1) & " = " & valorObjetivo & " Valor encontrado en " & num_iteraciones & " validaciones de formula", vbInformation
End Sub





<font color = "blue">**Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo**</font><br>

Ninguna

<font color = "blue">**Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño**</font><br>

En el archivo EXCEL adjunto se puede ver que el algoritmo implementado logra encontrar en un promedio de 70 evaluaciones cualquier digito dentro del rango por la tecnica de Divide y Venceras, y con un SUPER TOTAL de 10311 evaluaciones para calcular TODOS lso digitos del rango <br>

Ver gráfica de numero de evaluaciones pra cada DIGITO OBJETIVO